# 01 Convert models from various frameworks
* [1. Caffe to ONNX](#1.-Caffe-to-ONNX)
* [2. Keras to ONNX](#2.-Keras-to-ONNX)
* [3. TensorFlow to ONNX](#3.-TensorFlow-to-ONNX)
* [4. PyTorch to ONNX](#4.-PyTorch-to-ONNX)
* [5. Caffe2 to ONNX](#5.-Caffe2-to-ONNX)
* [6. MXNet to ONNX](#6.-MXNet-to-ONNX)

In [1]:
!pip install -q onnx==1.7.0 coremltools onnxmltools 

## 1. Caffe to ONNX
https://github.com/BVLC/caffe/wiki/Model-Zoo

In [2]:
# # 獲得預訓練模型 (mobilenet)
# !wget -q https://github.com/shicai/MobileNet-Caffe/blob/master/mobilenet.caffemodel?raw=true -O Caffe/model.caffemodel
# !wget -q https://raw.githubusercontent.com/shicai/MobileNet-Caffe/master/mobilenet_deploy.prototxt -O Caffe/model.prototxt

In [3]:
import onnx
import coremltools
import onnxmltools

In [4]:
caffe_path = "Caffe/model.caffemodel"
proto_path = "Caffe/model.prototxt"
onnx_path = "Caffe/model.onnx"

In [5]:
coreml_model = coremltools.converters.caffe.convert((caffe_path, proto_path))
onnx_model = onnxmltools.convert_coreml(coreml_model)
onnx.checker.check_model(onnx_model)
onnx.save(onnx_model, onnx_path)

## 2. Keras to ONNX
https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [6]:
# # 獲得預訓練模型 (mobilenet)，並將模型存成 h5 檔
# from tensorflow.keras.applications.mobilenet import MobileNet
# model = MobileNet(weights='imagenet', include_top=True)
# model.save('Keras/model.h5') 

### Use the onnxmltools modoule to convert the model

In [7]:
import onnx
import onnxmltools
from tensorflow.keras.models import load_model

In [8]:
keras_path = 'Keras/model.h5'
onnx_path = 'Keras/model.onnx'

In [9]:
keras_model = load_model(keras_path)
onnx_model = onnxmltools.convert_keras(keras_model)
onnx.checker.check_model(onnx_model)
onnx.save(onnx_model, onnx_path)

tf executing eager_mode: True
INFO:keras2onnx:tf executing eager_mode: True
tf.keras model eager_mode: False
INFO:keras2onnx:tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 248 -> 63
INFO:keras2onnx:The ONNX operator number change on the optimization: 248 -> 63


### Use the keras2onnx modoule to convert the model

In [10]:
!pip install -q keras2onnx

In [11]:
import onnx
import keras2onnx
from tensorflow.keras.models import load_model

In [12]:
keras_path = 'Keras/model.h5'
onnx_path = 'Keras/model.onnx'

In [13]:
keras_model = load_model(keras_path)
onnx_model = keras2onnx.convert_keras(keras_model)
onnx.checker.check_model(onnx_model)
keras2onnx.save_model(onnx_model, onnx_path)

tf executing eager_mode: True
INFO:keras2onnx:tf executing eager_mode: True
tf.keras model eager_mode: False
INFO:keras2onnx:tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 248 -> 63
INFO:keras2onnx:The ONNX operator number change on the optimization: 248 -> 63


## 3. TensorFlow to ONNX
https://github.com/tensorflow/models/tree/master/research/slim#pre-trained-models

In [14]:
# # 獲得預訓練模型 (tensorflow 1.x mobilenet)
# !wget -P TensorFlow http://download.tensorflow.org/models/mobilenet_v1_2018_02_22/mobilenet_v1_1.0_224.tgz
# !tar zxvf TensorFlow/mobilenet_v1_1.0_224.tgz -C TensorFlow/

In [15]:
# # 獲得預訓練模型 (tensorflow 2.x mobilenet)
# import tensorflow as tf
# from tensorflow.keras.applications.mobilenet import MobileNet
# model = MobileNet(weights='imagenet', include_top=True)
# # Save to SavedModel directory
# tf.saved_model.save(model, 'TensorFlow/saved_model') 

In [16]:
!pip install -q -U tf2onnx

### Convert with SavedModel

In [17]:
!python -m tf2onnx.convert \
        --saved-model ./TensorFlow/saved_model \
        --output ./TensorFlow/model.onnx

2020-10-29 18:11:58,247 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2020-10-29 18:12:03,685 - INFO - Signatures found in model: [serving_default].
2020-10-29 18:12:03,686 - WARNING - '--signature_def' not specified, using first signature: serving_default
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2020-10-29 18:12:04,530 - WARNING - From /opt/conda/lib/python3.7/site-packages/tf2onnx/tf_loader.py:413: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2020-10-29 18:12:05,451 - INFO - Using tensorflow=2.0.0, onnx=1.7.0, tf2onnx=1.7.1/796841
2020-10-29 18:12:05,451 - INFO - Using opset <onnx, 8>
2020-10-29 18:12:06,269 - INFO - Computed 0 values for constant folding
2020-10-29 18:12:08,485 - INFO - Optimizing ONNX model
2020-10-29 18:12:09,180 - INFO - After optimization: BatchNormalizat

### Convert with frozen graph

In [18]:
!head TensorFlow/mobilenet_v1_1.0_224_info.txt

Model: mobilenet_v1_1.0_224
Input: input
Output: MobilenetV1/Predictions/Reshape_1


In [19]:
!python -m tf2onnx.convert \
        --input ./TensorFlow/mobilenet_v1_1.0_224_frozen.pb \
        --inputs input:0 \
        --outputs MobilenetV1/Predictions/Reshape_1:0 \
        --output ./TensorFlow/model.onnx

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
2020-10-29 18:12:12,869 - WARNING - From /opt/conda/lib/python3.7/site-packages/tf2onnx/tf_loader.py:146: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2020-10-29 18:12:12,870 - WARNING - From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/graph_util_impl.py:275: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 0 variables.
2020-10-29 18:12:12,890 - INFO - Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
2020-10-29 18:12:12,913 - INFO 

## 4. PyTorch to ONNX
https://pytorch.org/docs/stable/torchvision/models.html

In [20]:
# !pip install torch==1.4.0 torchvision==0.5.0

In [21]:
# # 獲得預訓練模型 (mobilenet_v2)，並將模型存成 pth 檔
# model = torchvision.models.mobilenet_v2(pretrained=True)
# torch.save(model.state_dict(), 'PyTorch/model.pth')

In [22]:
import onnx
import torch.onnx
print(torch.__version__)
import torchvision

1.4.0


In [23]:
torch_path = "PyTorch/model.pth"
onnx_path = "PyTorch/model.onnx"

In [24]:
dummy_input = torch.randn(1, 3, 224, 224)
model = torchvision.models.mobilenet_v2(pretrained=False)
state_dict = torch.load(torch_path, map_location='cpu')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [25]:
torch.onnx.export(model, dummy_input, onnx_path)
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

## 5. Caffe2 to ONNX
https://github.com/facebookarchive/models

In [26]:
# # # 獲得預訓練模型 (squeezenet)
#!cd Caffe2; python -m caffe2.python.models.download squeezenet

In [27]:
import onnx
from caffe2.python.onnx.frontend import caffe2_net_to_onnx_model
from caffe2.proto import caffe2_pb2

In [28]:
predict_path = "Caffe2/predict_net.pb"
init_path = "Caffe2/init_net.pb"
onnx_path = "Caffe2/model.onnx"

In [29]:
data_type = onnx.TensorProto.FLOAT
data_shape = (1, 3, 224, 224)
value_info = {"data": (data_type, data_shape)}

In [30]:
predict_net = caffe2_pb2.NetDef()
with open(predict_path, "rb") as f:
    predict_net.ParseFromString(f.read())
    
init_net = caffe2_pb2.NetDef()
with open(init_path, "rb") as f:
    init_net.ParseFromString(f.read())

In [31]:
if predict_net.name == "":
    predict_net.name = "ModelNameHere"

In [32]:
onnx_model = caffe2_net_to_onnx_model(
    predict_net,
    init_net,
    value_info,
)
onnx.checker.check_model(onnx_model)
onnx.save(onnx_model, onnx_path)

## 6. MXNet to ONNX
https://cv.gluon.ai/model_zoo/classification.html

In [33]:
# 安裝完 onnx 1.2.1 版本後須重啟 kernel 
!pip install -q onnx==1.2.1
!pip install -q mxnet --pre

ERROR: tf2onnx 1.7.1 has requirement onnx>=1.4.1, but you'll have onnx 1.2.1 which is incompatible.


In [34]:
# # 獲得預訓練模型 (resnet18)
# import mxnet as mx
# path='http://data.mxnet.io/models/imagenet/'
# [mx.test_utils.download(path+'resnet/18-layers/resnet-18-0000.params', 
#                         fname='model-0000.params', dirname='MXNet'),
#  mx.test_utils.download(path+'resnet/18-layers/resnet-18-symbol.json',
#                         fname='model-symbol.json', dirname='MXNet')]

In [35]:
import onnx
print(onnx.__version__)
import mxnet as mx
import numpy as np
from mxnet.contrib import onnx as onnx_mxnet

1.2.1


In [36]:
sym_path = 'MXNet/model-symbol.json'
params_path = 'MXNet/model-0000.params'
onnx_path = 'MXNet/model.onnx'

In [37]:
input_shape = (1,3,224,224)
converted_model_path = onnx_mxnet.export_model(
    sym_path, 
    params_path, 
    [input_shape], 
    np.float32, 
    onnx_path
)

In [38]:
# Load onnx model and check it
onnx_model = onnx.load(converted_model_path)
onnx.checker.check_model(onnx_model)